In [1]:
!python --version
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

Python 3.6.8 :: Anaconda, Inc.


In [8]:
df = pd.read_csv('bioactivity_data.csv')
df = df[df.canonical_smiles.notna()]
df_no_smiles = df.drop(columns='canonical_smiles')
smiles = []
for i in df.canonical_smiles.tolist():
    cpd = str(i).split('.')
    cpd_longest = max(cpd, key=len)
    smiles.append(cpd_longest)
    
smiles = pd.Series(smiles, name='canonical_smiles')
df_clean_smiles = pd.concat([df_no_smiles, smiles], axis=1)
df_clean_smiles

,molecule_chembl_id,standard_value,bioactivity_class,canonical_smiles
0,CHEMBL9650,30.0,active,CCC(C)[C@H](NC(=O)[C@@H]1CCCN1[P@@](=O)(OC)[C@...
1,CHEMBL273396,93.0,active,CCC(C)[C@H](NC(=O)[C@@H]1CCCN1[P@@](=O)(OC)[C@...
2,CHEMBL108102,42.0,active,COC(=O)N[C@H](C(=O)N[C@@H](Cc1ccccc1)C(O)CN(Cc...
3,CHEMBL322033,1000.0,active,COC(=O)N(C)C(C(=O)N[C@@H](Cc1ccccc1)[C@H](O)CN...
4,CHEMBL20143,4.0,active,CC(C)[C@H](NC(=O)OCc1ccccc1)C(=O)NC(Cc1ccc(OCc...
...,...,...,...,...
3174,CHEMBL376817,5100.0,NaN,NaN
3175,CHEMBL224644,5300.0,NaN,NaN
3176,CHEMBL114,0.4,NaN,NaN
3177,CHEMBL405969,290.0,NaN,NaN


## Calculate Lipinski descriptors
Chistopher Lipinski came up with a set of rule-of-thumb for evaluating the druglikeness of compounds.
This is based on the Absorbtion, Distribution, Metabolism and Excretion (ADME) that is also know as the pharmokinetic profile.
Lipinski analyzed all orally active FDA-approved drugs in the formulation of what is know as the Rule-Of-Five or Lipinski's-Rule which states the following:

- Molecular weight < 500 Dalton
- Octanol-water partition coefficient (LogP) < 5
- Hydrogen bond donors < 5
- Hydrogen bond acceptors < 10

In [15]:
def lipinski(smiles, verbose=False):
    moldata=[]
    for elem in smiles:
        mol = Chem.MolFromSmiles(elem)
        moldata.append(mol)
        
    baseData = np.arange(1,1)
    i=0
    for mol in moldata:
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
        
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])
        
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i += 1
        
    columnNames=['MW', 'LogP', 'NumHDonors', 'NumHAcceptors']
    descriptors = pd.DataFrame(data=baseData, columns=columnNames)
    
    return descriptors

In [19]:
df_lipinski = lipinski(df.canonical_smiles)

TypeError: 'numpy.int32' object is not callable

In [20]:
print(df_lipinski.size)
print(df.size)

12296
12296


In [23]:
df_combined = pd.concat([df, df_lipinski], axis=1)
df_combined.head()

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors
0,CHEMBL9650,CCC(C)[C@H](NC(=O)[C@@H]1CCCN1[P@@](=O)(OC)[C@...,30.0,active,694.811,3.0471,5.0,8.0
1,CHEMBL273396,CCC(C)[C@H](NC(=O)[C@@H]1CCCN1[P@@](=O)(OC)[C@...,93.0,active,694.811,3.0471,5.0,8.0
2,CHEMBL108102,COC(=O)N[C@H](C(=O)N[C@@H](Cc1ccccc1)C(O)CN(Cc...,42.0,active,617.719,2.5085,5.0,8.0
3,CHEMBL322033,COC(=O)N(C)C(C(=O)N[C@@H](Cc1ccccc1)[C@H](O)CN...,1000.0,active,633.831,3.4339,3.0,8.0
4,CHEMBL20143,CC(C)[C@H](NC(=O)OCc1ccccc1)C(=O)NC(Cc1ccc(OCc...,4.0,active,681.733,4.6737,4.0,7.0


## Convert IC50 to pIC50
To allow IC<sub>50</sub> data to be more uniformly distributed, we will convert IC<sub>50</sub> to the negative logarithmic scale which is essentially -log10(IC<sub>50</sub>).

This custom pIC50() will accept a Dataframe as input and will:
- Take the IC<sub>50</sub> values from the standard_value column and converts it from nM to M by multiplying the value by 10^-9
- Take the molar value and apply -log10
- Delete the standard_value column and create a new pIC50 column

In [24]:
def pIC50(input):
    pIC50=[]
    
    for i in input['standard_value_norm']:
        molar = i*(10**-9)
        pIC50.append(-np.log10(molar))
        
    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', 1)
    
    return x

In [26]:
df_combined.standard_value.describe()

count    3.074000e+03
mean     1.812257e+04
std      1.835036e+05
min      1.400000e-03
25%      2.200000e+00
50%      3.830000e+01
75%      8.500000e+02
max      5.000000e+06
Name: standard_value, dtype: float64

In [28]:
def norm_value(input):
    norm = []

    for i in input['standard_value']:
        if i > 100000000:
            i = 100000000
        norm.append(i)

    input['standard_value_norm'] = norm
    x = input.drop('standard_value', 1)
        
    return x

In [29]:
df_norm = norm_value(df_combined)
df_norm.head()

,molecule_chembl_id,canonical_smiles,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,standard_value_norm
0,CHEMBL9650,CCC(C)[C@H](NC(=O)[C@@H]1CCCN1[P@@](=O)(OC)[C@...,active,694.811,3.0471,5.0,8.0,30.0
1,CHEMBL273396,CCC(C)[C@H](NC(=O)[C@@H]1CCCN1[P@@](=O)(OC)[C@...,active,694.811,3.0471,5.0,8.0,93.0
2,CHEMBL108102,COC(=O)N[C@H](C(=O)N[C@@H](Cc1ccccc1)C(O)CN(Cc...,active,617.719,2.5085,5.0,8.0,42.0
3,CHEMBL322033,COC(=O)N(C)C(C(=O)N[C@@H](Cc1ccccc1)[C@H](O)CN...,active,633.831,3.4339,3.0,8.0,1000.0
4,CHEMBL20143,CC(C)[C@H](NC(=O)OCc1ccccc1)C(=O)NC(Cc1ccc(OCc...,active,681.733,4.6737,4.0,7.0,4.0


In [30]:
df_combined.standard_value.describe()

count    3.074000e+03
mean     1.812257e+04
std      1.835036e+05
min      1.400000e-03
25%      2.200000e+00
50%      3.830000e+01
75%      8.500000e+02
max      5.000000e+06
Name: standard_value, dtype: float64

In [31]:
df_final = pIC50(df_norm)
df_final.head()

,molecule_chembl_id,canonical_smiles,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,CHEMBL9650,CCC(C)[C@H](NC(=O)[C@@H]1CCCN1[P@@](=O)(OC)[C@...,active,694.811,3.0471,5.0,8.0,7.522879
1,CHEMBL273396,CCC(C)[C@H](NC(=O)[C@@H]1CCCN1[P@@](=O)(OC)[C@...,active,694.811,3.0471,5.0,8.0,7.031517
2,CHEMBL108102,COC(=O)N[C@H](C(=O)N[C@@H](Cc1ccccc1)C(O)CN(Cc...,active,617.719,2.5085,5.0,8.0,7.376751
3,CHEMBL322033,COC(=O)N(C)C(C(=O)N[C@@H](Cc1ccccc1)[C@H](O)CN...,active,633.831,3.4339,3.0,8.0,6.000000
4,CHEMBL20143,CC(C)[C@H](NC(=O)OCc1ccccc1)C(=O)NC(Cc1ccc(OCc...,active,681.733,4.6737,4.0,7.0,8.397940


In [32]:
df_final.pIC50.describe()

count    3074.000000
mean        7.320122
std         1.715855
min         2.301030
25%         6.070581
50%         7.416815
75%         8.657577
max        11.853872
Name: pIC50, dtype: float64

In [34]:
#Remove intermediate bioactivity class and save
df_final = df_final[df_final['bioactivity_class'] != 'intermediate']
df_final.to_csv('hiv_bioactivitydata_pIC50.csv')

## EDA